# [Chapter 3. Numbers, Dates, and Times](http://chimera.labs.oreilly.com/books/1230000000393/ch03.html)

Performing mathematical calculations with integers and floating-point numbers is easy in Python.  
However, if you need to perform calculations with fractions, arrays, or dates and times, a bit more work is required.  
The focus of this chapter is on such topics.

## [Rounding Numerical Values](http://chimera.labs.oreilly.com/books/1230000000393/ch03.html#_rounding_numerical_values)

### Problem

You want to round a floating-point number to a fixed number of decimal places.

### Solution

For simple rounding, use the built-in `round(value, ndigits)` function.

In [1]:
print(round(1.23, 1))
print(round(1.27, 1))
print(round(-1.23, 1))
print(round(1.25361, 3))

1.2
1.3
-1.2
1.254


When a value is exactly halfway between two choices, the behavior of round is to round to the nearest even digit.  
That is, values such as 1.5 or 2.5 both get rounded to 2.  
The number of digits given to `round()` can be negative, in which case rounding takes place for tens, hundreds, thousands, and so on.

In [2]:
a = 1627731
print(round(a, -1))
print(round(a, -2))
print(round(a, -3))

1627730
1627700
1628000


### Discussion

Don't confuse rounding with formatting a value for output.  
If your goal is simply to output a numerical value with a certain number of decimal places, you don't usually need to use `round()`.  
Instead, just specify the desired precision when formatting, like this:

In [3]:
x = 1.23456
print(format(x, '0.2f'))
print(format(x, '0.3f'))
print('Value is {:0.3f}'.format(x))

1.23
1.235
Value is 1.235


Also, resist the urge to round floating-point numbers to "fix" perceived accuracy problems.  
Don't do this:

In [4]:
a = 2.1
b = 4.2
c = a + b
print(c)
# "Fix" result?:
c = round(c, 2)
print(c)

6.300000000000001
6.3


For most applications involving floating point, it’s simply not necessary (or recommended) to do this.  
Although there are small errors introduced into calculations, the behavior of those errors are understood and tolerated.  
If avoiding such errors is important (e.g., in financial applications, perhaps), consider the use of the `decimal` module, which is discussed in the next recipe.

## [Performing Accurate Decimal Calculations](http://chimera.labs.oreilly.com/books/1230000000393/ch03.html#_performing_accurate_decimal_calculations)

### Problem

You need to perform accurate calculations with decimal numbers, and you don't want the small errors that naturally occur with floats.

### Solution

One issue with floating-point numbers is that they can't accurately represent all base-10 decimals.  
Moreover, even simple mathematical calculations introduce small errors.

In [5]:
a = 4.2
b = 2.1
print(a + b)
print((a + b) == 6.3)

6.300000000000001
False


These errors are a "feature" of the underlying CPU and the IEEE 754 arithmetic performed by its floating-point unit.  
Since Python's float data type stores data using the native representation, there's nothing you can do to avoid such errors if you write your code using `float` instances.

If you want more accuracy (and are willing to give up some performance), you can use the `decimal` module:

In [6]:
from decimal import Decimal

a = Decimal('4.2')
b = Decimal('2.1')
print(a + b)
print((a + b) == Decimal('6.3'))

6.3
True


At first glance, it might look a little wierd (i.e., specifying numbers as strings).  
However, `Decimal` objects work in every way that you would expect them to (like supporting all of the usual math operations, and so on).  
If you print them or use them in string formatting functions, they look like normal numbers.

A major feature of `decimal` is that it allows you to control different aspects of calculations, including number of digits and rounding.  
To do this, you create a local context and change its settings.

In [7]:
from decimal import localcontext

a = Decimal('1.3')
b = Decimal('1.7')
print(a / b)

0.7647058823529411764705882353


In [8]:
with localcontext() as ctx:
    ctx.prec = 3
    print(a / b)

0.765


In [9]:
with localcontext() as ctx:
    ctx.prec = 50
    print(a / b)

0.76470588235294117647058823529411764705882352941176


### Discussion

The decimal module implements [IBM’s "General Decimal Arithmetic Specification."](http://speleotrove.com/decimal/) Needless to say, there are a huge number of configuration options that are beyond the scope of this book.  
Newcomers to Python might be inclined to use the decimal module to work around perceived accuracy problems with the float data type.  
However, it’s really important to understand your application domain.  
If you’re working with science or engineering problems, computer graphics, or most things of a scientific nature, it’s simply more common to use the normal floating-point type.  
For one, very few things in the real world are measured to the 17 digits of accuracy that floats provide.  
Thus, tiny errors introduced in calculations just don’t matter.  
Second, the performance of native floats is significantly faster -- something that’s important if you’re performing a large number of calculations.  
That said, you can’t ignore the errors completely.  
Mathematicians have spent a lot of time studying various algorithms, and some handle errors better than others.  
You also have to be a little careful with effects due to things such as [subtractive cancellation](https://ece.uwaterloo.ca/~dwharder/NumericalAnalysis/02Numerics/Weaknesses/#howto) and adding large and small numbers together.

In [10]:
nums = [1.23e+18, 1, -1.23e+18]
# Watch, ladies and gentleman, as the 1 magically disappears:
sum(nums)

0.0

This latter example can be addressed by using a more accurate implementation in math.fsum():

In [11]:
import math
math.fsum(nums)

1.0

However, for other algorithms, you really need to study the algorithm and understand its error propagation properties.  
All of this said, the main use of the decimal module is in programs involving things such as finance.  
In such programs, it is extremely annoying to have small errors creep into the calculation.  
Thus, decimal provides a way to avoid that.  
It is also common to encounter `Decimal` objects when Python interfaces with databases -- again, especially when accessing financial data.

## [Formatting Numbers for Output](http://chimera.labs.oreilly.com/books/1230000000393/ch03.html#_formatting_numbers_for_output)

### Problem

You need to format a number for output, controlling the number of digits, alignment, inclusion of a thousands separator, and other details.

### Solution

To format a single number for output, use the built-in `format()` function.

In [12]:
x = 1234.56789
# Two decimal places of accuracy:
format(x, '0.2f')

'1234.57'

In [13]:
# Right justified in 10 characters, one-digit accuracy:
format(x, '>10.1f')

'    1234.6'

In [14]:
# Left justified:
format(x, '<10.1f')

'1234.6    '

In [15]:
# Centered:
format(x, '^10.1f')

'  1234.6  '

In [16]:
# Inclusion of thousands separator:
format(x, ',')

'1,234.56789'

In [17]:
format(x, '0,.1f')

'1,234.6'

If you want to use exponential notation, change the `f` to an `E` or `e`, depending on the case you want used for the exponential specifier.

In [18]:
format(x, 'e')

'1.234568e+03'

In [19]:
format(x, '0.2E')

'1.23E+03'

The general form of the width and precision in both cases is `'[<>^]?width[,]?(.digits)?'` where width and digits are integers and `?` signifies optional parts.  
The same format codes are also used in the `.format()` method of strings.

In [20]:
'The value is {:0,.2f}'.format(x)

'The value is 1,234.57'

### Discussion

Formatting numbers for output is usually straightforward.  
The technique shown works for both floating-point numbers and `Decimal` numbers in the `decimal` module.  
When the number of digits is restricted, values are rounded away according to the same rules of the `round()` function.

In [21]:
x

1234.56789

In [22]:
format(x, '0.1f')

'1234.6'

In [23]:
format(-x, '0.1f')

'-1234.6'

Formatting of values with a thousands separator is not locale aware.  
If you need to take that into account, you might investigate functions in the `locale` module.  
You can also swap separator characters using the `translate()` method of strings.

In [24]:
swap_separators = { ord('.'):',', ord(','):'.'}
format(x, ',').translate(swap_separators)

'1.234,56789'

In a lot of Python code, numbers are formatted using the % operator.

In [25]:
'%0.2f' % x

'1234.57'

In [26]:
'%10.1f' % x

'    1234.6'

In [27]:
'%-10.1f' % x

'1234.6    '

This formatting is still acceptable, but less powerful than the more modern `format()` method.  
For example, some features (e.g., adding thousands separators) aren’t supported when using the `%` operator to format numbers.

## [Working with Binary, Octal, and Hexadecimal Integers](http://chimera.labs.oreilly.com/books/1230000000393/ch03.html#_working_with_binary_octal_and_hexadecimal_integers)

### Problem

You need to convert or output integers represented by binary, octal, or hexadecimal digits.

### Solution

To convert an integer into a binary, octal, or hexadecimal text string, use the `bin()`, `oct()`, or `hex()` functions, respectively.

In [28]:
x = 1234
bin(x)

'0b10011010010'

In [29]:
oct(x)

'0o2322'

In [30]:
hex(x)

'0x4d2'

Alternatively, you can use the `format()` function if you don't want the `0b`, `0o`, or `0x` prefixes to appear.

In [31]:
format(x, 'b')

'10011010010'

In [32]:
format(x, 'o')

'2322'

In [33]:
format(x, 'x')

'4d2'

Integers are signed, so if you are working with negative numbers, the output will also include a sign.

In [34]:
x = -1234
format(x, 'b')

'-10011010010'

In [35]:
format(x, 'x')

'-4d2'

If you need to produce an unsigned bit value instead, you'll need to add in the maximum value to set the bit length.  
For example, to show a 32-bit value, use the following:

In [36]:
x = -1234
format(2**32 + x, 'b')

'11111111111111111111101100101110'

In [37]:
format(2**32 + x, 'x')

'fffffb2e'

To convert integer strings in different bases, simply use the `int()` function with an appropriate base.

In [38]:
int('4d2', 16)

1234

In [39]:
int('10011010010', 2)

1234

### Discussion

For the most part, working with binary, octal, and hexadecimal integers is straightforward.  
Just remember that these conversions only pertain to the conversion of integers to and from a textual representation.  
Under the covers, there’s just one integer type.  
Finally, there is one caution for programmers who use octal.  
The Python syntax for specifying octal values is slightly different than many other languages.  
For example, if you try something like this, you’ll get a syntax error:

Make sure you prefix the octal value with 0o, as shown here:

## [Packing and Unpacking Large Integers from Bytes](http://chimera.labs.oreilly.com/books/1230000000393/ch03.html#_packing_and_unpacking_large_integers_from_bytes)

### Problem

You have a byte string and you need to unpack it into an integer value.  
Alternatively, you need to convert a large integer into a byte string.

### Solution

Suppose your program needs to work with a 16-element byte string that holds a 128-bit integer value.

In [40]:
data = b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

To interpret the bytes as an integer, use `int.from_bytes()`, and specify the byte ordering like this:

In [41]:
len(data)

16

In [42]:
int.from_bytes(data, 'little')

69120565665751139577663547927094891008

In [43]:
int.from_bytes(data, 'big')

94522842520747284487117727783387188

To convert a large integer value back into a byte string, use the `int.to_bytes()` method, specifying the number of bytes and the byte order.

In [44]:
x = 94522842520747284487117727783387188
x.to_bytes(16, 'big')

b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

In [45]:
x.to_bytes(16, 'little')

b'4\x00#\x00\x01\xef\xcd\x00\xab\x90x\x00V4\x12\x00'

### Discussion

Converting large integer values to and from byte strings is not a common operation.  
However, it sometimes arises in certain application domains, such as cryptography or networking.  
For instance, IPv6 network addresses are represented as 128-bit integers.  
If you are writing code that needs to pull such values out of a data record, you might face this problem.  
As an alternative to this recipe, you might be inclined to unpack values using the `struct` module, as described in ["Reading and Writing Binary Arrays of Structures"](http://chimera.labs.oreilly.com/books/1230000000393/ch06.html#_problem_104).  
This works, but the size of integers that can be unpacked with `struct` is limited.  
Thus, you would need to unpack multiple values and combine them to create the final value.

In [46]:
data

b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

In [47]:
import struct

hi, lo = struct.unpack('>QQ', data)
print(hi)
print(lo)

5124093560524971
57965157801984052


In [48]:
(hi << 64) + lo

94522842520747284487117727783387188

The specification of the byte order (`little` or `big`) just indicates whether the bytes that make up the integer value are listed from the least to most significant or the other way around.  
This is easy to view using a carefully crafted hexadecimal value:

In [49]:
x = 0x01020304
x.to_bytes(4, 'big')

b'\x01\x02\x03\x04'

In [50]:
x.to_bytes(4, 'little')

b'\x04\x03\x02\x01'

If you try to pack an integer into a byte string, but it won't fit, you'll get an error.  
You can use the `int.bit_length()` method to determine how many bits are required to store a value if needed:

In [52]:
x = 523 ** 23
x

335381300113661875107536852714019056160355655333978849017944067

In [54]:
x.bit_length()

208

In [55]:
nbytes, rem = divmod(x.bit_length(), 8)
print(nbytes)
print(rem)

26
0


In [56]:
if rem:
    nbytes += 1

In [57]:
x.to_bytes(nbytes, 'little')

b'\x03X\xf1\x82iT\x96\xac\xc7c\x16\xf3\xb9\xcf\x18\xee\xec\x91\xd1\x98\xa2\xc8\xd9R\xb5\xd0'

## [Performing Complex-Valued Math](http://chimera.labs.oreilly.com/books/1230000000393/ch03.html#_performing_complex_valued_math)